In [1]:
from sdv.datasets.demo import download_demo

real_data, metadata = download_demo(
    modality='single_table',
    dataset_name='fake_hotel_guests'
)

In [4]:
real_data

,guest_email,has_rewards,room_type,amenities_fee,checkin_date,checkout_date,room_rate,billing_address,credit_card_number
0,michaelsanders@shaw.net,False,BASIC,37.89,27 Dec 2020,29 Dec 2020,131.23,"49380 Rivers Street\nSpencerville, AK 68265",4075084747483975747
1,randy49@brown.biz,False,BASIC,24.37,30 Dec 2020,02 Jan 2021,114.43,"88394 Boyle Meadows\nConleyberg, TN 22063",180072822063468
2,webermelissa@neal.com,True,DELUXE,0.00,17 Sep 2020,18 Sep 2020,368.33,"0323 Lisa Station Apt. 208\nPort Thomas, LA 82585",38983476971380
3,gsims@terry.com,False,BASIC,NaN,28 Dec 2020,31 Dec 2020,115.61,"77 Massachusetts Ave\nCambridge, MA 02139",4969551998845740
4,misty33@smith.biz,False,BASIC,16.45,05 Apr 2020,NaN,122.41,"1234 Corporate Drive\nBoston, MA 02116",3558512986488983
...,...,...,...,...,...,...,...,...,...
495,laurabennett@jones-duncan.net,False,BASIC,8.71,04 Jan 2021,06 Jan 2021,103.25,"5678 Office Road\nSan Francisco, CA 94103",3505516387300030
496,johnny71@cook.info,False,BASIC,16.31,24 Aug 2020,26 Aug 2020,115.81,"953 White Island\nChristopherside, TN 91366",2224524502892552
497,ygarcia@ballard-lopez.net,False,BASIC,30.59,11 Nov 2020,13 Nov 2020,141.61,"5678 Office Road\nSan Francisco, CA 94103",180096250673548
498,thomasdale@hall.com,False,BASIC,1.93,16 Jul 2020,18 Jul 2020,136.92,"5678 Office Road\nSan Francisco, CA 94103",4488223821722


In [12]:
filtered = real_data.drop(["guest_email", "checkin_date", "checkout_date", "billing_address", "credit_card_number"], axis=1)

In [13]:
filtered

,has_rewards,room_type,amenities_fee,room_rate
0,False,BASIC,37.89,131.23
1,False,BASIC,24.37,114.43
2,True,DELUXE,0.00,368.33
3,False,BASIC,NaN,115.61
4,False,BASIC,16.45,122.41
...,...,...,...,...
495,False,BASIC,8.71,103.25
496,False,BASIC,16.31,115.81
497,False,BASIC,30.59,141.61
498,False,BASIC,1.93,136.92


In [14]:
import sdv
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

In [15]:
metadata = SingleTableMetadata()

In [16]:
metadata.detect_from_dataframe(data=filtered)

In [17]:
python_dict = metadata.to_dict()

In [18]:
python_dict

{'columns': {'has_rewards': {'sdtype': 'boolean'},
  'room_type': {'sdtype': 'categorical'},
  'amenities_fee': {'sdtype': 'numerical'},
  'room_rate': {'sdtype': 'numerical'}},
 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1'}

In [19]:
synthesizer = CTGANSynthesizer(
    metadata,
    enforce_rounding=False,
    verbose=True
)

In [20]:
synthesizer.fit(filtered)

Epoch 1, Loss G:  3.6718,Loss D:  0.0034
Epoch 2, Loss G:  3.8899,Loss D:  0.0015
Epoch 3, Loss G:  3.7511,Loss D: -0.0057
Epoch 4, Loss G:  3.8135,Loss D: -0.0092
Epoch 5, Loss G:  3.7917,Loss D: -0.0133
Epoch 6, Loss G:  3.9172,Loss D: -0.0160
Epoch 7, Loss G:  3.6631,Loss D: -0.0034
Epoch 8, Loss G:  3.7780,Loss D: -0.0230
Epoch 9, Loss G:  3.4602,Loss D: -0.0101
Epoch 10, Loss G:  3.8050,Loss D: -0.0149
Epoch 11, Loss G:  3.6216,Loss D: -0.0284
Epoch 12, Loss G:  3.6613,Loss D: -0.0272
Epoch 13, Loss G:  3.6371,Loss D: -0.0199
Epoch 14, Loss G:  3.8309,Loss D: -0.0366
Epoch 15, Loss G:  3.7370,Loss D: -0.0258
Epoch 16, Loss G:  3.5938,Loss D: -0.0370
Epoch 17, Loss G:  3.6087,Loss D: -0.0393
Epoch 18, Loss G:  3.6459,Loss D: -0.0262
Epoch 19, Loss G:  3.7784,Loss D: -0.0179
Epoch 20, Loss G:  3.5719,Loss D: -0.0375
Epoch 21, Loss G:  3.5756,Loss D: -0.0501
Epoch 22, Loss G:  3.6401,Loss D: -0.0399
Epoch 23, Loss G:  3.6393,Loss D: -0.0268
Epoch 24, Loss G:  3.3396,Loss D: -0.0163
E

In [21]:
filtered.shape

(500, 4)

In [22]:
synthetic_data = synthesizer.sample(
    num_rows=500
)

In [23]:
from sdmetrics.single_column import KSComplement
from sdmetrics.reports import utils
from sdmetrics.single_column import StatisticSimilarity
from sdmetrics.reports.single_table import QualityReport
from sdmetrics.single_table import GMLogLikelihood
from sdmetrics.timeseries import LSTMDetection

In [24]:
filtered.columns

Index(['has_rewards', 'room_type', 'amenities_fee', 'room_rate'], dtype='object')

In [29]:
KSComplement.compute(real_data=filtered["has_rewards"], synthetic_data=synthetic_data[["has_rewards"]])

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [28]:
filtered = filtered.fillna(filtered.mean())
filtered

/tmp/ipykernel_1810907/100812235.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  filtered = filtered.fillna(filtered.mean())


,has_rewards,room_type,amenities_fee,room_rate
0,False,BASIC,37.890000,131.23
1,False,BASIC,24.370000,114.43
2,True,DELUXE,0.000000,368.33
3,False,BASIC,18.176066,115.61
4,False,BASIC,16.450000,122.41
...,...,...,...,...
495,False,BASIC,8.710000,103.25
496,False,BASIC,16.310000,115.81
497,False,BASIC,30.590000,141.61
498,False,BASIC,1.930000,136.92


In [27]:
synthetic_data

,has_rewards,room_type,amenities_fee,room_rate
0,False,BASIC,25.567481,222.046320
1,False,DELUXE,15.446154,111.665848
2,False,BASIC,24.102764,124.772915
3,False,BASIC,15.239827,186.460620
4,False,BASIC,22.359954,116.947321
...,...,...,...,...
495,False,BASIC,29.770210,155.938666
496,False,DELUXE,9.933602,199.682110
497,False,BASIC,18.378769,160.358022
498,False,BASIC,27.087690,204.224946


In [36]:
metadata = {
    "columns": {
        "has_rewards": {
            "sdtype": "boolean"
        },
        "room_type": {
            "sdtype": "categorical"
        },
        "amenities_fee": {
            "sdtype": "numerical"
        },
        "room_rate": {
            "sdtype": "numerical"
        }
    }
}

In [31]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    filtered,
    synthetic_data,
    metadata
)

Creating report: 100%|██████████| 4/4 [00:00<00:00, 131.10it/s]


Overall Quality Score: 79.07%

Properties:
Column Shapes: 84.53%
Column Pair Trends: 73.62%


In [32]:
report = QualityReport()

In [37]:
report.generate(filtered, synthetic_data, metadata)

Creating report: 100%|██████████| 4/4 [00:00<00:00, 69.66it/s]


Overall Quality Score: 79.07%

Properties:
Column Shapes: 84.53%
Column Pair Trends: 73.62%


In [38]:
report.get_score()

0.790749998868555

In [39]:
report.get_properties()

,Property,Score
0,Column Shapes,0.845345
1,Column Pair Trends,0.736155


In [40]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Quality Score
0,amenities_fee,KSComplement,0.625381
1,room_rate,KSComplement,0.866000
2,has_rewards,TVComplement,0.978000
3,room_type,TVComplement,0.912000


In [43]:
filtered

,has_rewards,room_type,amenities_fee,room_rate
0,False,BASIC,37.890000,131.23
1,False,BASIC,24.370000,114.43
2,True,DELUXE,0.000000,368.33
3,False,BASIC,18.176066,115.61
4,False,BASIC,16.450000,122.41
...,...,...,...,...
495,False,BASIC,8.710000,103.25
496,False,BASIC,16.310000,115.81
497,False,BASIC,30.590000,141.61
498,False,BASIC,1.930000,136.92
